## ABSTRACT

In this assignment, we are converting a SQL relational database into a non-relational NoSQL database. We have used MongoDB to convert from relational to a non-relational database. This dataset contains all the tables related to the ICC Cricket world cup. Some tables have been previously scraped using Beautiful soup in Python. Additional tables have been web scraped which contain various tweets corresponding to the trending hashtags during the world cup.

## Importing Libraries

In [24]:
import tweepy
import csv
import json
import re
import pymongo
import pandas as pd
import twitter

#Using pymongo library to import Mongodb
from pymongo import MongoClient
myclient = pymongo.MongoClient("mongodb://localhost:27017/") #27017 is the default poet to connect to MongoDB
mydb = myclient["test2"]
mycol = mydb["ICC"]

#reading sample data
df_players = pd.read_csv("ICC_CWC_19_Players.csv")
df_teams = pd.read_csv("ICC_CWC_19_Teams2.csv")
df_boards = pd.read_csv("ICC_CWC_19_Boards.csv")

## SAMPLE DATA

This is the sample data which we have used in previous assignments and is present in our relational SQL database

In [25]:
pd.set_option('display.max_rows',1000)
df_players.head(152)

,Player_Name,Player_Id,Team_Id
0,Gulbadin Naib (c),1,T10
1,Rashid Khan (vc),2,T10
2,Aftab Alam,3,T10
3,Asghar Afghan,4,T10
4,Dawlat Zadran,5,T10
5,Hamid Hassan,6,T10
6,Hashmatullah Shahidi,7,T10
7,Hazratullah Zazai,8,T10
8,Mohammad Nabi,9,T10
9,Mohammad Shahzad (wk),10,T10


In [26]:
df_teams

,Team_ID,Team_name,Matches,Wins,Losses,Board_Id
0,T1,India,10,7,2,B1
1,T2,Australia,10,7,3,B2
2,T3,England,10,7,3,B3
3,T4,New Zealand,10,6,3,B4
4,T5,Pakistan,8,5,3,B5
5,T6,Sri Lanka,8,3,4,B6
6,T7,South Africa,9,3,5,B7
7,T8,Bangladesh,9,3,5,B8
8,T9,West Indies,9,2,6,B9
9,T10,Afghanistan,9,0,9,B10


In [27]:
df_boards

,Board_Id,Board Name,Coach Name
0,B1,Board of Control for Cricket in India,Ravi Shastri
1,B2,Cricket Australia,Justin Langer
2,B3,England and Wales Cricket Board,Trevor Bayliss
3,B4,New Zealand Cricket,Gary Stead
4,B5,Pakistan Cricket Board,Mickey Arthur
5,B6,Sri Lanka Cricket,Chandika Hathurusingha
6,B7,Cricket South Africa,Ottis Gibson
7,B8,Bangladesh Cricket Board,Steve Rhodes
8,B9,Cricket West Indies,Floyd Reifer
9,B10,Afghanistan Cricket Board,Phil Simmons


## Web Scrape tweets from Twitter 

We have used Tweepy as a library to web scrape tweets. The hashtags selected are mostly those which were trending during World Cup 2019

In [28]:
#Twitter authorization codes
CONSUMER_KEY = 'auTIVP4hHiJA7Snb4XYWWEofR'
CONSUMER_SECRET = 'VeoKqZ1twMgYaxTSNB6MRO6dvg112BYuTajEZJMw8gzutjpDKR'
ACCESS_TOKEN = '1121864797-GyTUHsApFDjZGoP4ovCDXudSq48wXUXGiBwVczo'
ACCESS_TOKEN_SECRET = 'I5FK5vwn70xskQTgHema1VzAZLoHZIR1NDlTXHfvuTSVS'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth)

#Create a new csv file 
csv_file = open('cwc19final.csv', 'w+')

#Write rows in those csv files
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Timestamp', 'Tweet', 'username', 'hashtags' ,"followers_count"])

#Search for the mentioned hashtag
for tweet in tweepy.Cursor(api.search, q="#cwc19final", 
                          lang="en",
                          since="2019-01-01").items(100):
            csv_writer.writerow([tweet.created_at, tweet.text.encode('utf-8'), tweet.user.screen_name.encode('utf-8'), 
            [e['text'] for e in tweet._json['entities']['hashtags']] ,tweet.user.followers_count])

In [29]:
#Twitter authorization codes
CONSUMER_KEY = 'auTIVP4hHiJA7Snb4XYWWEofR'
CONSUMER_SECRET = 'VeoKqZ1twMgYaxTSNB6MRO6dvg112BYuTajEZJMw8gzutjpDKR'
ACCESS_TOKEN = '1121864797-GyTUHsApFDjZGoP4ovCDXudSq48wXUXGiBwVczo'
ACCESS_TOKEN_SECRET = 'I5FK5vwn70xskQTgHema1VzAZLoHZIR1NDlTXHfvuTSVS'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth)

#Create a new csv file 
csv_file = open('davidwarner.csv', 'w+')

#Write rows in those csv files
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Timestamp', 'Tweet', 'username', 'hashtags' ,"followers_count"])

#Search for the mentioned hashtag
for tweet in tweepy.Cursor(api.search, q="#davidwarner", 
                          lang="en",
                          since="2019-01-01").items(100):
            csv_writer.writerow([tweet.created_at, tweet.text.encode('utf-8'), tweet.user.screen_name.encode('utf-8'), [e['text'] for e in tweet._json['entities']['hashtags']] ,tweet.user.followers_count])

In [30]:
#Twitter authorization codes
CONSUMER_KEY = 'auTIVP4hHiJA7Snb4XYWWEofR'
CONSUMER_SECRET = 'VeoKqZ1twMgYaxTSNB6MRO6dvg112BYuTajEZJMw8gzutjpDKR'
ACCESS_TOKEN = '1121864797-GyTUHsApFDjZGoP4ovCDXudSq48wXUXGiBwVczo'
ACCESS_TOKEN_SECRET = 'I5FK5vwn70xskQTgHema1VzAZLoHZIR1NDlTXHfvuTSVS'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth)

#Create a new csv file 
csv_file = open('cwc19.csv', 'w+')

#Write rows in those csv file
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Timestamp', 'Tweet', 'username', 'hashtags' ,"followers_count"])

#Search for the mentioned hashtag
for tweet in tweepy.Cursor(api.search, q="#cwc19", 
                          lang="en",
                          since="2019-01-01").items(100):
            csv_writer.writerow([tweet.created_at, tweet.text.encode('utf-8'), tweet.user.screen_name.encode('utf-8'), [e['text'] for e in tweet._json['entities']['hashtags']] ,tweet.user.followers_count])

In [31]:
CONSUMER_KEY = 'auTIVP4hHiJA7Snb4XYWWEofR'
CONSUMER_SECRET = 'VeoKqZ1twMgYaxTSNB6MRO6dvg112BYuTajEZJMw8gzutjpDKR'
ACCESS_TOKEN = '1121864797-GyTUHsApFDjZGoP4ovCDXudSq48wXUXGiBwVczo'
ACCESS_TOKEN_SECRET = 'I5FK5vwn70xskQTgHema1VzAZLoHZIR1NDlTXHfvuTSVS'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth)

#Create a new csv file 
csv_file = open('kohli.csv', 'w+')

#Write rows in those csv files
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Timestamp', 'Tweet', 'username', 'hashtags' ,"followers_count"])

#Search for the mentioned hashtag
for tweet in tweepy.Cursor(api.search, q="#kohli", 
                          lang="en",
                          since="2019-01-01").items(100):
            csv_writer.writerow([tweet.created_at, tweet.text.encode('utf-8'), tweet.user.screen_name.encode('utf-8'), [e['text'] for e in tweet._json['entities']['hashtags']] ,tweet.user.followers_count])

In [32]:
CONSUMER_KEY = 'auTIVP4hHiJA7Snb4XYWWEofR'
CONSUMER_SECRET = 'VeoKqZ1twMgYaxTSNB6MRO6dvg112BYuTajEZJMw8gzutjpDKR'
ACCESS_TOKEN = '1121864797-GyTUHsApFDjZGoP4ovCDXudSq48wXUXGiBwVczo'
ACCESS_TOKEN_SECRET = 'I5FK5vwn70xskQTgHema1VzAZLoHZIR1NDlTXHfvuTSVS'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth)

#Create a new csv file 
csv_file = open('rohitsharma.csv', 'w+')

#Write rows in those csv files
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Timestamp', 'Tweet', 'username', 'hashtags' ,"followers_count"])

#Search for the mentioned hashtag
for tweet in tweepy.Cursor(api.search, q="#rohitsharma", 
                          lang="en",
                          since="2019-01-01").items(100):
            csv_writer.writerow([tweet.created_at, tweet.text.encode('utf-8'), tweet.user.screen_name.encode('utf-8'), [e['text'] for e in tweet._json['entities']['hashtags']] ,tweet.user.followers_count])

In [33]:
CONSUMER_KEY = 'auTIVP4hHiJA7Snb4XYWWEofR'
CONSUMER_SECRET = 'VeoKqZ1twMgYaxTSNB6MRO6dvg112BYuTajEZJMw8gzutjpDKR'
ACCESS_TOKEN = '1121864797-GyTUHsApFDjZGoP4ovCDXudSq48wXUXGiBwVczo'
ACCESS_TOKEN_SECRET = 'I5FK5vwn70xskQTgHema1VzAZLoHZIR1NDlTXHfvuTSVS'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth)

#Create a new csv file 
csv_file = open('teamaustralia.csv', 'w+')

#Write rows in those csv files
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Timestamp', 'Tweet', 'username', 'hashtags' ,"followers_count"])

#Search for the mentioned hashtag
for tweet in tweepy.Cursor(api.search, q="#teamaustralia", 
                          lang="en",
                          since="2019-01-01").items(100):
            csv_writer.writerow([tweet.created_at, tweet.text.encode('utf-8'), tweet.user.screen_name.encode('utf-8'), [e['text'] for e in tweet._json['entities']['hashtags']] ,tweet.user.followers_count])

In [34]:
CONSUMER_KEY = 'auTIVP4hHiJA7Snb4XYWWEofR'
CONSUMER_SECRET = 'VeoKqZ1twMgYaxTSNB6MRO6dvg112BYuTajEZJMw8gzutjpDKR'
ACCESS_TOKEN = '1121864797-GyTUHsApFDjZGoP4ovCDXudSq48wXUXGiBwVczo'
ACCESS_TOKEN_SECRET = 'I5FK5vwn70xskQTgHema1VzAZLoHZIR1NDlTXHfvuTSVS'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth)

#Create a new csv file 
csv_file = open('teamindia.csv', 'w+')

#Write rows in those csv files
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Timestamp', 'Tweet', 'username', 'hashtags' ,"followers_count"])

#Search for the mentioned hashtag
for tweet in tweepy.Cursor(api.search, q="#teamindia", 
                          lang="en",
                          since="2019-01-01").items(100):
            csv_writer.writerow([tweet.created_at, tweet.text.encode('utf-8'), tweet.user.screen_name.encode('utf-8'), [e['text'] for e in tweet._json['entities']['hashtags']] ,tweet.user.followers_count])

In [35]:
CONSUMER_KEY = 'auTIVP4hHiJA7Snb4XYWWEofR'
CONSUMER_SECRET = 'VeoKqZ1twMgYaxTSNB6MRO6dvg112BYuTajEZJMw8gzutjpDKR'
ACCESS_TOKEN = '1121864797-GyTUHsApFDjZGoP4ovCDXudSq48wXUXGiBwVczo'
ACCESS_TOKEN_SECRET = 'I5FK5vwn70xskQTgHema1VzAZLoHZIR1NDlTXHfvuTSVS'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth)

#Create a new csv file 
csv_file = open('worldcup.csv', 'w+')

#Write rows in those csv files
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Timestamp', 'Tweet', 'username', 'hashtags' ,"followers_count"])

#Search for the mentioned hashtag
for tweet in tweepy.Cursor(api.search, q="#worldcup", 
                          lang="en",
                          since="2019-01-01").items(100):
            csv_writer.writerow([tweet.created_at, tweet.text.encode('utf-8'), tweet.user.screen_name.encode('utf-8'), [e['text'] for e in tweet._json['entities']['hashtags']] ,tweet.user.followers_count])

### Twitter API to scrape details of all players from twitter and printing them in a table

In [ ]:
CONSUMER_KEY = 'auTIVP4hHiJA7Snb4XYWWEofR'
CONSUMER_SECRET = 'VeoKqZ1twMgYaxTSNB6MRO6dvg112BYuTajEZJMw8gzutjpDKR'
OAUTH_TOKEN = '1121864797-GyTUHsApFDjZGoP4ovCDXudSq48wXUXGiBwVczo'
OAUTH_TOKEN_SECRET = 'I5FK5vwn70xskQTgHema1VzAZLoHZIR1NDlTXHfvuTSVS'

auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET, CONSUMER_KEY, CONSUMER_SECRET)

twitter_api = twitter.Twitter(auth=auth)
print(twitter_api)

auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET, CONSUMER_KEY, CONSUMER_SECRET)

twitter_api = twitter.Twitter(auth=auth)
print(twitter_api)

Top_ten_Test_Batsmen = ["@imVkohli", "@stevesmith49", "@marnus3cricket", "@NotNossy", "@davidwarner31",
                        "@cheteshwar1", "@babarazam258", "@ajinkyarahane88", "@root66", "@benstokes38"]

player_name = []
twitter_id = []
followers = []
friends = []
verify = []

#Using for loop to iterate and print all values
for i in Top_ten_Test_Batsmen:
    twitter_id.append(i)
    player_name.append(twitter_api.statuses.user_timeline(screen_name=i)[1]['user']['name'])
    followers.append(twitter_api.statuses.user_timeline(screen_name=i)[1]['user']['followers_count'])
    friends.append(twitter_api.statuses.user_timeline(screen_name=i)[1]['user']['friends_count'])
    verify.append(twitter_api.statuses.user_timeline(screen_name=i)[1]['user']['verified'])

Twitter_Specifications = pd.DataFrame({
             'Player Name': player_name,
             'Twitter ID': twitter_id,
             'Followers Count': followers,
             'Friends Count': friends,
             'Verified': verify})

Twitter_Specifications

### Script to convert structured database to MongoDB

This script reads the data in python and writes it in MongoDB using pymongo library. Nested for loops are used to create sub documents under a document from the SQL tables. Iterrows function has been used to create object for every row of the tables with matching IDs. First it iterates on the players dataframe, next iterates on the teams dataframe, then iterates on boards dataframe and lastly combines all the respective data into one object/document and creates sub-objects/sub-documents for MongoDB.
As MongoDB can contain sub documents under the documents, the following script has been written to convert from SQL schema to NoSQL schema.

In [36]:
for index,row in df_players.iterrows():
    t_id = row['Team_Id']
    team = ""
    team_object = {}
    board_object = {}
    for index1,row1 in df_teams.iterrows():
        if row1['Team_ID'] == t_id:
            
            # team 
            team = row1
            team_object = {
            "t_id": team.Team_ID,
            "teamname" : team.Team_name
            }   
            
            # board
            b_id = team.Board_Id
            board = ""
            for index2,row3 in df_boards.iterrows():
                if row3['Board_Id'] == b_id:
                    board = row3

            board_object = {
                "Board Id": board.Board_Id,
                "Board Name": board['Board Name'],
                "Coach Name": board['Coach Name']
            }

    ICC = {
        "PlayerID" : row['Player_Id'],
        "Player" : row['Player_Name'],
        
        "team" : team_object,
        "cricket_board": board_object 
    }
    mycol.insert_one(ICC)

In [38]:
#reading all the csv files
df_kohli_tweets = pd.read_csv("kohli.csv")
df_benstokes_tweets = pd.read_csv("benstokes.csv")
df_cwc19_tweets = pd.read_csv("cwc19.csv")
df_cwc19final_tweets = pd.read_csv("cwc19final.csv")
df_davidwarner_tweets = pd.read_csv("davidwarner.csv")
df_rohitsharma_tweets = pd.read_csv("rohitsharma.csv")
df_teamindia_tweets = pd.read_csv("teamindia.csv")
df_teamaustralia_tweets = pd.read_csv("teamaustralia.csv")
df_worldcup_tweets = pd.read_csv("worldcup.csv")
df_most_runs = pd.read_csv("most_runs.csv", encoding = "ISO-8859-1")
df_most_wickets = pd.read_csv("most_wickets.csv", encoding = "ISO-8859-1")

#converting csv files to a data frame and inserting those data frame into mongodb 
records1 = df_cwc19_tweets.to_dict(orient = 'records')
result = mydb.cwc19_tweets.insert_many(records1 )
records2 = df_cwc19final_tweets.to_dict(orient = 'records')
result = mydb.cwc29final_tweets.insert_many(records2 )
records3 = df_worldcup_tweets.to_dict(orient = 'records')
result = mydb.worldcup_tweets.insert_many(records3 )
records4 = df_teamindia_tweets.to_dict(orient = 'records')
result = mydb.teamindia_tweets.insert_many(records4 )
records5 = df_teamaustralia_tweets.to_dict(orient = 'records')
result = mydb.teamaustralia_tweets.insert_many(records5 )
records6 = df_kohli_tweets.to_dict(orient = 'records')
result = mydb.kohli_tweets.insert_many(records6 )
records7 = df_benstokes_tweets.to_dict(orient = 'records')
result = mydb.benstokes_tweets.insert_many(records7 )
records8 = df_rohitsharma_tweets.to_dict(orient = 'records')
result = mydb.rohitsharma_tweets.insert_many(records8 )
records9 = df_davidwarner_tweets.to_dict(orient = 'records')
result = mydb.davidwarner_tweets.insert_many(records9 )
records10 = Twitter_Specifications.to_dict(orient = 'records')
result = mydb.Twitter_Specifications.insert_many(records10)
records11 = df_most_runs.to_dict(orient = 'records')
result = mydb.most_runs.insert_many(records11)
records12 = df_most_wickets.to_dict(orient = 'records')
result = mydb.most_wickets.insert_many(records12)
records13 = df_teams.to_dict(orient = 'records')
result = mydb.teams_dataset.insert_many(records13)



## RUBRICS

### 1. What are tags are associated with a person, place or thing?

The various hashtags used in our domain are as follows:

 - #worldcup
 - #cwc19
 - #cwc19final
 - #teamindia
 - #teamaustralia
 - #kohli
 - #rohitsharma
 - #benstokes
 - #davidwarner


### 2. What social media users are like other social media users in your domain?


#### Using query to display the instance where players have social media account from ICC collection

> db.ICC.aggregate([{$lookup:{localField: "user_id", from : "Twitter_Specifications", foreignField: "_id", as: "TwitterHandle" }},{$project :{ "Player":1,"TwitterHandle.Twitter ID":1}}]).pretty();

#### The above query will return the following output:

{
	"Player" : "Steve Smith",
    
	"TwitterHandle" : {
		"Twitter ID" : "@stevesmith49"
	}
}
{
	"Player" : "David Warner",
    
	"TwitterHandle" : {
		"Twitter ID" : "@davidwarner31"
	}
    }
{ "Player" : "Joe Root", "TwitterHandle" : { "Twitter ID" : "@root66" } }
{
	"Player" : "Ben Stokes",
    
	"TwitterHandle" : {
		"Twitter ID" : "@benstokes38"
	}
}
{
	"Player" : "Virat Kohli (c)",
    
	"TwitterHandle" : {
		"Twitter ID" : "@imVkohli"
	}
}
{
	"Player" : "Kane Williamson (c)",
    
	"TwitterHandle" : {
		"Twitter ID" : "@NotNossy"
	}
}
{
	"Player" : "Babar Azam (vc)",
    
	"TwitterHandle" : {
		"Twitter ID" : "@babarazam258"
	}
}


### 3. What people, places or things are popular in your domain?

 - Popular people: Virat Kohli, Rohit Sharma, Benstokes, David Warner
 
	
 - Popular Teams: Team India, Team Australia


### 4. What people, places or things are trending in your domain? (A trend is popularity over time)

For a time period between March 2019 to July 2019, we observed all the trending players associated with the world cup tweets and noticed that Virat Kohli was tagged for the most number of times. Unfortunately since we used a developer account to download the tweets we could not download enough data to represent it





### 5. You must test that you can retrieve your data by implementing all of the use-cases in that you created in all    of the assignments so far.



#### Using query, we implemented 5 use-cases which are as below

#### 1. Use case which gives Highest Runs:
> db.most_runs.find().sort({Runs:-1}).limit(1).pretty()

#### The above query will give the following output: 
{
	"_id" : ObjectId("5e8e46fc8b48d32720feb2ab"),
    
	"Player" : "RG Sharma(INDIA)",
	"Mat" : 9,
	"Inns" : 9,
	"NO" : 1,
	"Runs" : 648,
	"HS" : "140",
	"Ave" : 81,
	"BF" : 659,
	"SR" : 98.33,
	"100" : 5,
	"50" : 1,
	"0" : 0,
	"4s" : 67,
	"6s" : 14
}

#### 2. Use-cases which give most number of wickets:
> db.most_wickets.find().sort({Wickets:-1}).limit(1).pretty()

#### The above query will return the following output:
{
	"_id" : ObjectId("5e8e46fc8b48d32720feb2dd"),
    
	"Player" : "MA StarcÊ(AUS)",
	"Mat" : 10,
	"Inns" : 10,
	"Overs" : 92.2,
	"Mdns" : 5,
	"Runs" : 502,
	"Wkts" : 27,
	"BBI" : 43977,
	"Ave" : 18.59,
	"Econ" : 5.43,
	"SR" : 20.5,
	"4" : 2,
	"5" : 2
}

#### 3. Use-case which gives players with average runs scored

> db.most_runs.aggregate({$match : {Runs : {$gt : 250, $lte : 350 }}}]).pretty()

#### The above query will return the following output:

{
	"_id" : ObjectId("5e8e46fc8b48d32720feb2bd"),
    
	"Player" : "LRPL TaylorÊ(NZ)",
	"Mat" : 10,
	"Inns" : 9,
	"NO" : 0,
	"Runs" : 350,
	"HS" : "82",
	"Ave" : 38.88,
	"BF" : 465,
	"SR" : 75.26,
	"100" : 0,
	"50" : 3,
	"0" : 0,
	"4s" : 29,
	"6s" : 2
}
{
	"_id" : ObjectId("5e8e46fc8b48d32720feb2be"),
    
	"Player" : "UT KhawajaÊ(AUS)",
	"Mat" : 9,
	"Inns" : 9,
	"NO" : 0,
	"Runs" : 316,
	"HS" : "89",
	"Ave" : 35.11,
	"BF" : 358,
	"SR" : 88.26,
	"100" : 0,
	"50" : 2,
	"0" : 0,
	"4s" : 30,
	"6s" : 1
}
{
	"_id" : ObjectId("5e8e46fc8b48d32720feb2bf"),
    
	"Player" : "JC ButtlerÊ(ENG)",
	"Mat" : 11,
	"Inns" : 9,
	"NO" : 0,
	"Runs" : 312,
	"HS" : "103",
	"Ave" : 34.66,
	"BF" : 254,
	"SR" : 122.83,
	"100" : 1,
	"50" : 2,
	"0" : 0,
	"4s" : 22,
	"6s" : 8
}
{
	"_id" : ObjectId("5e8e46fc8b48d32720feb2c0"),
    
	"Player" : "HE van der DussenÊ(SA)",
	"Mat" : 9,
	"Inns" : 6,
	"NO" : 1,
	"Runs" : 311,
	"HS" : "95",
	"Ave" : 62.2,
	"BF" : 344,
	"SR" : 90.4,
	"100" : 0,
	"50" : 3,
	"0" : 0,
	"4s" : 14,
	"6s" : 10
}
{
	"_id" : ObjectId("5e8e46fc8b48d32720feb2c1"),
    
	"Player" : "Imam-ul-HaqÊ(PAK)",
	"Mat" : 8,
	"Inns" : 8,
	"NO" : 0,
	"Runs" : 305,
	"HS" : "100",
	"Ave" : 38.12,
	"BF" : 400,
	"SR" : 76.25,
	"100" : 1,
	"50" : 1,
	"0" : 0,
	"4s" : 31,
	"6s" : 1
}
{
	"_id" : ObjectId("5e8e46fc8b48d32720feb2c2"),
    
	"Player" : "Q de Kock(SA)",
	"Mat" : 9,
	"Inns" : 9,
	"NO" : 1,
	"Runs" : 305,
	"HS" : "68",
	"Ave" : 38.12,
	"BF" : 351,
	"SR" : 86.89,
	"100" : 0,
	"50" : 3,
	"0" : 0,
	"4s" : 34,
	"6s" : 4
}
{
	"_id" : ObjectId("5e8e46fc8b48d32720feb2c3"),
    
	"Player" : "SD Hope(WI)",
	"Mat" : 9,
	"Inns" : 8,
	"NO" : 0,
	"Runs" : 274,
	"HS" : "96",
	"Ave" : 34.25,
	"BF" : 389,
	"SR" : 70.43,
	"100" : 0,
	"50" : 3,
	"0" : 0,
	"4s" : 21,
	"6s" : 3
}
{
	"_id" : ObjectId("5e8e46fc8b48d32720feb2c4"),
    
	"Player" : "MDKJ Perera(SL)",
	"Mat" : 7,
	"Inns" : 7,
	"NO" : 0,
	"Runs" : 273,
	"HS" : "78",
	"Ave" : 39,
	"BF" : 246,
	"SR" : 110.97,
	"100" : 0,
	"50" : 3,
	"0" : 0,
	"4s" : 32,
	"6s" : 1
}
{
	"_id" : ObjectId("5e8e46fc8b48d32720feb2c5"),
    
	"Player" : "MS Dhoni(INDIA)",
	"Mat" : 9,
	"Inns" : 8,
	"NO" : 2,
	"Runs" : 273,
	"HS" : "56*",
	"Ave" : 45.5,
	"BF" : 311,
	"SR" : 87.78,
	"100" : 0,
	"50" : 2,
	"0" : 0,
	"4s" : 20,
	"6s" : 5
}
{
	"_id" : ObjectId("5e8e46fc8b48d32720feb2c6"),
    
	"Player" : "SO Hetmyer(WI)",
	"Mat" : 9,
	"Inns" : 8,
	"NO" : 1,
	"Runs" : 257,
	"HS" : "54",
	"Ave" : 36.71,
	"BF" : 253,
	"SR" : 101.58,
	"100" : 0,
	"50" : 2,
	"0" : 0,
	"4s" : 25,
	"6s" : 6
}
{
	"_id" : ObjectId("5e8e46fc8b48d32720feb2c7"),
    
	"Player" : "Rahmat Shah(AFG)",
	"Mat" : 9,
	"Inns" : 9,
	"NO" : 0,
	"Runs" : 254,
	"HS" : "62",
	"Ave" : 28.22,
	"BF" : 390,
	"SR" : 65.12,
	"100" : 0,
	"50" : 1,
	"0" : 1,
	"4s" : 31,
	"6s" : 1
}
{
	"_id" : ObjectId("5e8e46fc8b48d32720feb2c8"),
    
	"Player" : "Mohammad Hafeez(PAK)",
	"Mat" : 8,
	"Inns" : 8,
	"NO" : 0,
	"Runs" : 253,
	"HS" : "84",
	"Ave" : 31.62,
	"BF" : 285,
	"SR" : 88.77,
	"100" : 0,
	"50" : 1,
	"0" : 0,
	"4s" : 22,
	"6s" : 5
}

#### 4. Use case to give highest trending player on twitter

> db.Twitter_Specifications.aggregate([{$sort:{"Followers Count" : -1}},{$project:{"_id": 0 ,"Player Name":1, "Followers Count":1}}]).pretty()

#### The above query will give the following output: 

{ "Player Name" : "Virat Kohli", "Followers Count" : 34733316 }

{ "Player Name" : "Ajinkya Rahane", "Followers Count" : 5786164 }

{ "Player Name" : "cheteshwar pujara", "Followers Count" : 2382612 }

{ "Player Name" : "David Warner", "Followers Count" : 2040967 }

{ "Player Name" : "Steve Smith", "Followers Count" : 1721864 }

{ "Player Name" : "Babar Azam", "Followers Count" : 1398324 }

{ "Player Name" : "Joe Root", "Followers Count" : 741020 }

{ "Player Name" : "Ben Stokes", "Followers Count" : 738291 }

{ "Player Name" : "Kane Williamson", "Followers Count" : 42662 }

{ "Player Name" : "Marnus Labuschagne", "Followers Count" : 5058 }

#### 5. Use-case to display team india players

> db.ICC.aggregate([{$match:{"team.teamname" :"India" }},{$project:{"_id":0,"PlayerID":1, "Player":1, "team.teamname":1}}]).pretty()

#### The above query will give the following output:

{
	"PlayerID" : 62,
	
    "Player" : "Virat Kohli (c)",
	"team" : {
		"teamname" : "India"
	}
}
{
	"PlayerID" : 63,
	
    "Player" : "Rohit Sharma (vc)",
	"team" : {
		"teamname" : "India"
	}
}
{
	"PlayerID" : 64,
	
    "Player" : "Shikhar Dhawan",
	"team" : {
		"teamname" : "India"
	}
}
{
	"PlayerID" : 65,
    
	"Player" : "K. L. Rahul",
	"team" : {
		"teamname" : "India"
	}
}
{
	"PlayerID" : 66,
    
	"Player" : "Vijay Shankar",
	"team" : {
		"teamname" : "India"
	}
}
{
	"PlayerID" : 67,
    
	"Player" : "MS Dhoni (wk)",
	"team" : {
		"teamname" : "India"
	}
}
{
	"PlayerID" : 68,
    
	"Player" : "Kedar Jadhav",
	"team" : {
		"teamname" : "India"
	}
}
{
	"PlayerID" : 69,
    
	"Player" : "Dinesh Karthik",
	"team" : {
		"teamname" : "India"
	}
}
{
	"PlayerID" : 70,
    
	"Player" : "Yuzvendra Chahal",
	"team" : {
		"teamname" : "India"
	}
}
{
	"PlayerID" : 71,
    
	"Player" : "Kuldeep Yadav",
	"team" : {
		"teamname" : "India"
	}
}
{
	"PlayerID" : 72,
    
	"Player" : "Bhuvneshwar Kumar",
	"team" : {
		"teamname" : "India"
	}
}
{
	"PlayerID" : 73,
    
	"Player" : "Jasprit Bumrah",
	"team" : {
		"teamname" : "India"
	}
}
{
	"PlayerID" : 74,
    
	"Player" : "Hardik Pandya",
	"team" : {
		"teamname" : "India"
	}
}
{
	"PlayerID" : 75,
    
	"Player" : "Ravindra Jadeja",
	"team" : {
		"teamname" : "India"
	}
}
{
	"PlayerID" : 76,
    
	"Player" : "Mohammed Shami",
	"team" : {
		"teamname" : "India"
	}
}


### 6. A description of the design choices you made in converting your SQL Schema that makes sense. 

 - Our relational database had 3 tables namely, the Players, Teams and Boards which were gathered from the 2019 Cricket World Cup.


 - The player table and Team table were joined through the team_id key and the Boards and team table were joined using the board_id key.
 
 
 - To convert the above database into a non-relational database, each player name was considered to be a document and the team table and player table were inserted as a sub document. The aggregate of all player documents were considered as a collection.

### AUDIT VALIDITY/ACCURACY

The acquired data is said to be valid when it doesn’t contain any null values. Using Python, we checked for null values and duplicate values in the data. Only the top 10 rows were selected from all 3 sources and was found to have no null values. The rest of the unwanted rows were removed since the ranking were missing, making the final data valid and accurate.

### AUDIT COMPLETENESS

The data collected were obtained from genuine websites and are real world data. The data is said to be complete when the obtained result matches the real time data. The final data which we have obtained correlates with ICC world raking, and hence proves to be complete

### AUDIT CONSISTENCY/UNIFORMITY

The data is consistent with no null values. The data collected from 3 different sources are linked through by a common attribute.

### REPORT

#### Libraries used:

import tweepy

import csv

import json

import re

import pymongo

import pandas as pd

import twitter

#### files scrapped:
1. benstokes
2. cwc19
3. cwc19final
4. davidwarner
5. kohli
6. rohitsharma
7. teamaustralia
8. teamindia
9. worldcup

#### files used in previous assignments:
1. ICC_CWC_19_Players.csv
2. ICC_CWC_19_Teams2.csv
3. ICC_CWC_19_Boards.csv

### Code used:

##### 1. Web Scrape all player hashtags from twitter
CONSUMER_KEY = 'auTIVP4hHiJA7Snb4XYWWEofR'

CONSUMER_SECRET = 'VeoKqZ1twMgYaxTSNB6MRO6dvg112BYuTajEZJMw8gzutjpDKR'

ACCESS_TOKEN = '1121864797-GyTUHsApFDjZGoP4ovCDXudSq48wXUXGiBwVczo'

ACCESS_TOKEN_SECRET = 'I5FK5vwn70xskQTgHema1VzAZLoHZIR1NDlTXHfvuTSVS'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)

auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth)

#Create a new csv file 

csv_file = open('teamindia.csv', 'w+')

#Write rows in those csv files

csv_writer = csv.writer(csv_file)

csv_writer.writerow(['Timestamp', 'Tweet', 'username', 'hashtags' ,"followers_count"])

#Search for the mentioned hashtag

for tweet in tweepy.Cursor(api.search, q="#teamindia", 
                          
                          lang="en",
                          
                          since="2019-01-01").items(100):
            
            csv_writer.writerow([tweet.created_at, tweet.text.encode('utf-8'), tweet.user.screen_name.encode('utf-8'), [e['text'] for e in tweet._json['entities']['hashtags']] ,tweet.user.followers_count])
            
#### 2. Web scrape when twitter name is specified

Top_ten_Test_Batsmen = ["@imVkohli", "@stevesmith49", "@marnus3cricket", "@NotNossy", "@davidwarner31",
                        "@cheteshwar1", "@babarazam258", "@ajinkyarahane88", "@root66", "@benstokes38"]

player_name = []

twitter_id = []

followers = []

friends = []

verify = []

#Using for loop to iterate and print all values

for i in Top_ten_Test_Batsmen:
    
    twitter_id.append(i)
    
    player_name.append(twitter_api.statuses.user_timeline(screen_name=i)[1]['user']['name'])
    
    followers.append(twitter_api.statuses.user_timeline(screen_name=i)[1]['user']['followers_count'])
    
    friends.append(twitter_api.statuses.user_timeline(screen_name=i)[1]['user']['friends_count'])
    
    verify.append(twitter_api.statuses.user_timeline(screen_name=i)[1]['user']['verified'])

Twitter_Specifications = pd.DataFrame({
             
             'Player Name': player_name,
             
             'Twitter ID': twitter_id,
             
             'Followers Count': followers,
             
             'Friends Count': friends,
             
             'Verified': verify})
             
#### 3. Script to convert structured database into MongoDB


for index,row in df_players.iterrows():
   
   t_id = row['Team_Id']
   
   team = ""
   
   team_object = {}
   
   board_object = {}
   
   for index1,row1 in df_teams.iterrows():
       
       if row1['Team_ID'] == t_id:
            
            # team 
            team = row1
            team_object = {
            "t_id": team.Team_ID,
            "teamname" : team.Team_name
            }   
            
            # board
            b_id = team.Board_Id
            board = ""
            for index2,row3 in df_boards.iterrows():
                if row3['Board_Id'] == b_id:
                    board = row3

            board_object = {
                "Board Id": board.Board_Id,
                "Board Name": board['Board Name'],
                "Coach Name": board['Coach Name']
            }

    ICC = {
        "PlayerID" : row['Player_Id'],
        "Player" : row['Player_Name'],
        
        "team" : team_object,
        "cricket_board": board_object 
    }
    mycol.insert_one(ICC)

4. To read CSV files into MongoDB

#reading all the csv files

df_kohli_tweets = pd.read_csv("kohli.csv")

df_benstokes_tweets = pd.read_csv("benstokes.csv")

df_cwc19_tweets = pd.read_csv("cwc19.csv")

df_cwc19final_tweets = pd.read_csv("cwc19final.csv")

df_davidwarner_tweets = pd.read_csv("davidwarner.csv")

df_rohitsharma_tweets = pd.read_csv("rohitsharma.csv")

df_teamindia_tweets = pd.read_csv("teamindia.csv")

df_teamaustralia_tweets = pd.read_csv("teamaustralia.csv")

df_worldcup_tweets = pd.read_csv("worldcup.csv")

df_most_runs = pd.read_csv("most_runs.csv", encoding = "ISO-8859-1")

df_most_wickets = pd.read_csv("most_wickets.csv", encoding = "ISO-8859-1")

#converting csv files to a data frame and inserting those data frame into mongodb 

records1 = df_cwc19_tweets.to_dict(orient = 'records')

result = mydb.cwc19_tweets.insert_many(records1 )

records2 = df_cwc19final_tweets.to_dict(orient = 'records')

result = mydb.cwc29final_tweets.insert_many(records2 )

records3 = df_worldcup_tweets.to_dict(orient = 'records')

result = mydb.worldcup_tweets.insert_many(records3 )

records4 = df_teamindia_tweets.to_dict(orient = 'records')

result = mydb.teamindia_tweets.insert_many(records4 )

records5 = df_teamaustralia_tweets.to_dict(orient = 'records')

result = mydb.teamaustralia_tweets.insert_many(records5 )

records6 = df_kohli_tweets.to_dict(orient = 'records')

result = mydb.kohli_tweets.insert_many(records6 )

records7 = df_benstokes_tweets.to_dict(orient = 'records')

result = mydb.benstokes_tweets.insert_many(records7 )

records8 = df_rohitsharma_tweets.to_dict(orient = 'records')

result = mydb.rohitsharma_tweets.insert_many(records8 )

records9 = df_davidwarner_tweets.to_dict(orient = 'records')

result = mydb.davidwarner_tweets.insert_many(records9 )

records10 = Twitter_Specifications.to_dict(orient = 'records')

result = mydb.Twitter_Specifications.insert_many(records10)

records11 = df_most_runs.to_dict(orient = 'records')

result = mydb.most_runs.insert_many(records11)

records12 = df_most_wickets.to_dict(orient = 'records')

result = mydb.most_wickets.insert_many(records12)

records13 = df_teams.to_dict(orient = 'records')

result = mydb.teams_dataset.insert_many(records13)

#### 5. query to display the instance where players have social media account from ICC collection

db.ICC.aggregate([{$lookup:{localField: "user_id", from : "Twitter_Specifications", foreignField: "_id", as: "TwitterHandle" }},{$project :{ "Player":1,"TwitterHandle.Twitter ID":1}}]).pretty();

#### 6. Use case which gives Highest Runs:
db.most_runs.find().sort({Runs:-1}).limit(1).pretty()

#### 7. Use-cases which give most number of wickets:
db.most_wickets.find().sort({Wickets:-1}).limit(1).pretty()

##### 8. Use-case which gives players with average runs scored
db.most_runs.aggregate({$match : {Runs : { 𝑔𝑡:250,gt:250,lte : 350 }}}]).pretty()

#### 9. Use case to give highest trending player on twitter
db.Twitter_Specifications.aggregate([{$sort:{"Followers Count" : -1}},{$project:{"_id": 0 ,"Player Name":1, "Followers Count":1}}]).pretty()

#### 10. Use-case to display team india players
db.ICC.aggregate([{$match:{"team.teamname" :"India" }},{$project:{"_id":0,"PlayerID":1, "Player":1, "team.teamname":1}}]).pretty()








### CONCLUSION   

Primary focus of this assignment is to convert a relational SQL database into a non-relational NOSQL database. There are various queries which were used to show data into a non-relational database

### CONTRIBUTION

Your contribution towards project. How much code did you write and how much you took from other site or some other source.

I contributed By Own: 40%

By External source: 40%

Provided by the professor : 20%

### CITATIONS

https://www.pythonforbeginners.com/

https://www.py4e.com/book.php 

https://www.w3schools.com/python/

https://youtu.be/ng2o98k983k

https://twitter.com/login?redirect_after_login=https%3A%2F%2Fdeveloper.twitter.com%2Fapps

https://docs.mongodb.com




### LICENSE

This is a human-readable summary of (and not a substitute for) the license. Disclaimer. 

You are free to: 

Share — copy and redistribute the material in any medium or format 
Adapt — remix, transform, and build upon the material for any purpose, even commercially. 
The licensor cannot revoke these freedoms if you follow the license terms. 


Under the following terms:

Attribution — You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use. 

No additional restrictions — You may not apply legal terms or technological measures that legally restrict others from doing anything the license permits. 

Notices:

You do not have to comply with the license for elements of the material in the public domain or where your use is permitted by an applicable exception or limitation. 

No warranties are given. The license may not give you all the permissions necessary for your intended use. For example, other rights such as publicity, privacy, or moral rights may limit how you use the material.

